In [47]:
# !pip install -q evaluate
# !pip install -q nlpaug
# !pip install -q transformers
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install accelerate
# ! pip install -U accelerate
# ! pip install -U transformers
# import torch, time
# import evaluate
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import nlpaug.augmenter.word as naw
# from datasets import Dataset, DatasetDict
# from sklearn.model_selection import train_test_split

# from transformers import AutoTokenizer
# from transformers import AutoModelForSequenceClassification
# from transformers import TrainingArguments, Trainer, TrainerCallback

In [2]:
# Trainer module based on pytorch , hence its manadatory , or we can also use keras to use trainer module
# Trainer module is from training , which aspects model,training set, validation, test dataset, tokenizer, hyperparametre, metrics to use from users etc
# This module returns / used for training , evalution, predictions

# when training transformer using Trainer should not be used when we are not using pytorch , as trainer module is based on pytorch
# If not using pytorch , we can train transformer using keras , hence transformer module not used
# for fine grained control over training  loop , we can use accelerate library

In [3]:
from datasets import load_dataset   # datasets is library having inbuild datasets compatible for transformer library
emotion_df = load_dataset("emotion")  #dataset of emotions , joy,anger etc. Multiclass calssification
emotion_df     # 16000 records for training data , In datasets library every row represented as dictionary , columns as key and there name as values

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
emotion_df['train'][0]  # first record from train df

{'text': 'i didnt feel humiliated', 'label': 0}

In [5]:
emotion_df['train'][15999]  # last record from train df  # text is data and label is column or Y

# Name of variables as "KEY" while records in it act as "values"

{'text': 'i know a lot but i feel so stupid because i can not portray it',
 'label': 0}

In [6]:
emotion_df['test'][0]   # first record of test dataframe

{'text': 'im feeling rather rotten so im not very ambitious right now',
 'label': 0}

In [7]:
# converting this dictionary format with pandas df
train_df = emotion_df['train'].to_pandas()
train_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [8]:
features = emotion_df['train'].features  # df.info() # MultiClass Classifiaction (6)classes
features # for datatypes of columns # its more like df.info() function in pandas

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [10]:
# which label id , shows which class among 6 classes
features['label'].int2str(5)  #0,1,2,3,4,5

'surprise'

In [11]:
# MAPPING OF EACH LABEL ID TO LABLE NAME  :
# useful when we do text classification using transformer , hence instead of giving output in int
# model return output in "Labels"

In [12]:
id2label = {idx:features['label'].int2str(idx) for idx in range(6)}
print(id2label)
label2id = {j:i for i,j in id2label.items() }
print(label2id)

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}


In [13]:
# Cheacking distribution of classes
train_df['label'].value_counts(normalize=True).sort_index() # Imbalaced dataset
# model will be good at predicting majorily represented classes , and will struggle a lot for underrepresented class like "5:surprise class"
# why not oversampling done while using transformers ?
#>> as deep learning model are very much good at memorisiing or discovring data, hence model may get biased in production for those duplicated class, or model may not generalized very
# well for testing points

#Instead of that we can modify loss function of model during training , hence we can incorporate distribution of classes in loss function so to give higher weights to underrepresented class

0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: label, dtype: float64

In [14]:
#Tokenize Data : Used MiniLm model for tokenize , not bertbase : 109million param ,81%performance slower, miniLM :66M param but still 80% performance faster

In [15]:
from transformers import AutoTokenizer
model_ckpt = "microsoft/MiniLM-L12-H384-uncased"   # model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [16]:
tokenizer(emotion_df['train']['text'][:1]) # encoding first record

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [17]:
emotion_df['train']['text'][0]

'i didnt feel humiliated'

In [18]:
# Tokenizer to all dataset (train,test, validate)
def tokenize_text(data):
  return tokenizer(data['text'],truncation= True,max_length=512)

In [19]:
emotion_df = emotion_df.map(tokenize_text,batched=True) #map is funtion of dataset library to aply finction to all data at a time
emotion_df

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [20]:
emotion_df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [21]:
len(train_df)

16000

In [22]:
train_df['label'].value_counts().sort_index()

0    4666
1    5362
2    1304
3    2159
4    1937
5     572
Name: label, dtype: int64

In [23]:
# Dealing with imbalaced data
class_weights = (1-(train_df['label'].value_counts().sort_index() / len(train_df))).values
class_weights
#calculating class weights based on the distribution of labels in your training dataset.
#These class weights are used to address the issue of imbalanced data when training a machine learning model, particularly in the context of multi-class classification
#  The purpose of subtracting this from 1 is to give more weight to underrepresented classes. In other words, the class weights will be higher for minority classes.
# and hence model does not get too bias in mahority classes

array([0.708375 , 0.664875 , 0.9185   , 0.8650625, 0.8789375, 0.96425  ])

In [24]:
1-(4666/16000)

0.708375

In [25]:
# as trainer module of transformer is based on pytorch , we ultimately need to convert this from numpy array to torch tensors
import torch
class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], device='cuda:0')

In [26]:
emotion_df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [27]:
train_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [28]:
# we have to tell trainer module which is our target column  : in our case its "label"
# In documentaion its written that :
# "your model can compute the loss if a labels argument is provided and that loss is returned as the first element of the tuple (if your model returns tuples)"
# refremce : https://huggingface.co/docs/transformers/main_classes/trainer

# Hence we should first change the name of our column from "label":"labels"
emotion_df = emotion_df.rename_columns({"label":"labels"})
emotion_df


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [29]:

# Now we can define trainer : here we will define custom trainer as we have to do or incorporate  custom loss computation for our imbalanced dataset
import torch
from torch import nn
from transformers import Trainer


class WeightedLossTrainer(Trainer):  # weighted Loss trainer will be our new class , which is subclass in Trainer module of transformer
  def compute_loss(self, model, inputs, return_outputs=False):

    outputs = model(**inputs)    # feeding input to model to have output
    logits = outputs.get("logits") # from this output we will calculate logits, which we will compare with label to compute loss
    labels = inputs.get("labels")  # extract labels
    loss_func = nn.CrossEntropyLoss(weight = class_weights)
    loss = loss_func(logits,labels)

    return (loss,outputs) if  return_outputs else loss



In [30]:
# Now we have to define Model , training dataset , Hyperparametres, Metrics
# initialize model
from transformers import AutoModelForSequenceClassification  # as we are using text classification , we use Sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=6,id2label=id2label,label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from sklearn.metrics import *

def compute_metrics(pred):
  labels = pred.label_ids   # extract
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels,preds,average="weighted")
  return {"f1":f1}

In [36]:
from transformers.training_args_tf import TrainingArguments
# Hyperparametre for model

from transformers import TrainingArguments

batch_size = 64
# log the training loss at each epoch
logging_steps = len(emotion_df['train']) // batch_size  #(16000//64)
output_dir = "/content/drive/MyDrive/Endpoint/output"                # which will save checkpoint of model,tokenizer, config details
training_args  = TrainingArguments(output_dir=output_dir,
                                   num_train_epochs=5,
                                   learning_rate=2e-5,
                                   per_device_train_batch_size=batch_size,
                                   per_device_eval_batch_size=batch_size,
                                   weight_decay=0.01,
                                   evaluation_strategy='epoch',
                                   logging_steps=logging_steps
                                  )




In [37]:
trainer = WeightedLossTrainer(model=model,
                              args=training_args,
                              compute_metrics=compute_metrics,
                              train_dataset=emotion_df['train'],
                              eval_dataset=emotion_df['validation'],
                              tokenizer = tokenizer)

In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.875200,0.636872,0.825957
2,0.562600,0.420710,0.892076
3,0.373100,0.323533,0.913325
4,0.295600,0.273972,0.925204
5,0.253200,0.260092,0.923270


TrainOutput(global_step=1250, training_loss=0.47193045959472657, metrics={'train_runtime': 264.6171, 'train_samples_per_second': 302.324, 'train_steps_per_second': 4.724, 'total_flos': 582422632630272.0, 'train_loss': 0.47193045959472657, 'epoch': 5.0})

In [42]:
test_predictions = trainer.predict(emotion_df['test'])  # giving test dataset

In [46]:
test_predictions

PredictionOutput(predictions=array([[ 4.1871977 , -0.5160435 , -1.6854371 , -0.14843364, -0.8620906 ,
        -1.2483866 ],
       [ 4.2184854 , -0.3603429 , -1.7263066 , -0.0912152 , -0.9670185 ,
        -1.3671473 ],
       [ 4.1936507 , -0.5746215 , -1.6283691 , -0.11323347, -0.9203863 ,
        -1.2202849 ],
       ...,
       [-0.06343261,  4.4418592 ,  0.11570144, -1.4685432 , -1.493565  ,
        -1.8257092 ],
       [-0.400466  ,  4.419962  ,  0.17453724, -1.5345982 , -1.2784909 ,
        -1.6675433 ],
       [-1.4914532 , -2.2622745 , -0.20299634, -0.4780376 ,  2.2489235 ,
         2.3617501 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 4]), metrics={'test_loss': 0.24704885482788086, 'test_f1': 0.9219446578827489, 'test_runtime': 2.2869, 'test_samples_per_second': 874.56, 'test_steps_per_second': 13.993})

In [44]:
predicted_labels = test_predictions.predictions.argmax(axis=1)  # Extract the predicted labels
predicted_labels

array([0, 0, 0, ..., 1, 1, 5])